In [13]:
%load_ext autoreload
%autoreload 2 
%load_ext autoreload
%autoreload 
#import sys
#for p in sys.path:
#    print(p)
#!jupyter kernelspec list
#!pip install networkx
#!pip install node2vec
import numpy as np
import math as math 
import tensorflow as tf
import graph_synthesis as gs
from random_mini_batches import random_mini_batches_perm
from random_mini_batches import random_mini_batches2
import networkx as nx
import node2vec as n2v
tf.test.gpu_device_name()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/device:GPU:0'

In [14]:
class FullNN:
    
    def __init__(self, inputdat, labeldat, testdat, testlabeldat, layer_dims, decayrate = 0.995,
                 l2reg = 1e-3, learning_rate = 0.008, batch_size = 256, epoch_nums = 20000):
        
        #Data
        self.inputdat = inputdat
        self.labeldat = labeldat
        self.testdat = testdat
        self.testlabeldat = testlabeldat
        self.num_eg = inputdat.shape[0]
        
        #Hyperparameters
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epoch_nums = epoch_nums
        self.l2reg = l2reg  
        self.decayrate = decayrate 
        
        #Initialize
        tf.reset_default_graph()
        self.initdata()
        
        #Create graph
        self.output = self.model()
        self.optimizer, self.cost = self.train()
        self.prediction, self.accuracy = self.test()
        self.summary_op = self.create_summaries()
        
        #Run Tensorflow
        config = tf.ConfigProto(allow_soft_placement = True, log_device_placement = True, device_count = {'GPU': 1})
        self.sess = tf.Session(config = config)
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        self.sess.run(init)
        
        #Tensorboard setup
        self.writer = tf.summary.FileWriter('./FullNNSave',  (self.sess).graph)
        
        #Train model 
        self.graph_classifier()
        self.prediction_num, self.error_num = self.evaluate("train") 
        
    def  create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.cost)
            tf.summary.scalar("accuracy", self.accuracy)
            tf.summary.histogram("histogram_loss", self.cost)
            summary_op = tf.summary.merge_all()
        return summary_op
    
    def initdata(self):
        self.tflearnrate = tf.placeholder(tf.float32, name = "learnrate")
        self.mode = tf.placeholder(tf.bool, name = "mode")
        self.input = tf.placeholder(tf.float32, shape = [None, self.layer_dims[0]], name = "input")
        self.label = tf.placeholder(tf.float32, shape = [None, 1], name = "label")

    
    def model(self):
        layer_dims = self.layer_dims
        layers = {}
        layers[0] = self.input
        for i in range(len(layer_dims)-1):
            if (i < len(layer_dims) - 2):
                scope = 'layers' + str(i+1)
                with tf.variable_scope(scope):
                    #layers[i+1] = tf.layers.dense(units = layer_dims[i+1], activation = tf.nn.relu, inputs = layers[i])
                    layers[i+1] = tf.contrib.layers.fully_connected(num_outputs = layer_dims[i+1], \
                                                        activation_fn = None, inputs = layers[i], scope = 'dense')
                    layers[i+1] = tf.contrib.layers.batch_norm(layers[i+1], is_training = self.mode, scope = 'batchnorm')
                    layers[i+1] = tf.nn.relu(layers[i+1], 'relu')
            else:
                scope = 'layers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[len(layer_dims)-1] = tf.layers.dense(units = layer_dims[len(layer_dims)-1], \
                                                    activation = tf.nn.sigmoid, inputs = layers[len(layer_dims)-2])
        
        output = layers[len(layer_dims)-1]
        return output 
    
    def train(self):
        #Saver for data 
        self.saver = tf.train.Saver()
        self.global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name = 'global_step')
        
        #Define cost 
        reg_loss = self.l2reg * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) + \
                                                                                                            reg_loss 
        
        #Batch Normalization
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.tflearnrate).minimize(cost, global_step = self.global_step)
            
        return optimizer, cost 
    
    def test(self):
        prediction = tf.greater(self.output, tf.constant(0.5, dtype = tf.float32))
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.cast(prediction, dtype = tf.float32), self.label)
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                tf.summary.scalar('accuracy', accuracy)
        return prediction, accuracy
    
    def graph_classifier(self):
        curlearnrate = self.learning_rate
        step = 0 
        for epochs in range(self.epoch_nums):
            output = []
            mini_batch = random_mini_batches_perm(self.inputdat, self.labeldat, self.batch_size)
            curlearnrate = curlearnrate * self.decayrate 
            for dat in mini_batch:
                step = step + 1 
                _, cost, output, summary = self.sess.run([self.optimizer, self.cost, self.output, self.summary_op], 
                                                feed_dict = {self.input: dat[0], self.label: dat[1], \
                                                             self.mode: True, self.tflearnrate: curlearnrate})
                (self.writer).add_summary(summary, global_step = step)
            if ((epochs + 1) % 500 == 0):
                _, errortrain = self.evaluate("train")
                cost, accuracy = self.sess.run([self.cost, self.accuracy], \
                        feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                     self.mode: False, self.tflearnrate: curlearnrate})
                (self.saver).save(self.sess, 'checkpoints/CNN', global_step=self.global_step)
                _, errortest = self.evaluate("test")
                print("epoch", (epochs+1), "    |    ", "cost", "%.6e" % cost, "    |    ", 
                      "error train", "%.3f" % (errortrain * 100), "    |    ", "error test", "%.3f" % (errortest * 100))
        
    def evaluate(self, mode):
        #Get predictions
        if mode == "train":
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                                 self.mode: False})
        else:
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.testdat, self.label: self.testlabeldat, \
                                                 self.mode: False})
        predictions = np.array(predictions).reshape((self.num_eg,1))
        predictions = predictions.astype(int)
        
        #Compute error
        error = np.sum((predictions - self.labeldat)**2)/self.num_eg
        #print("error " + mode, error * 100)
        
        return predictions, error

In [ ]:
def n2vdata(data, num_eg, n2vdim, vertex):
    n2vdata = np.zeros((num_eg, n2vdim, vertex))
    for g in range(num_eg):
        graph = nx.from_numpy_matrix(data["graphs"][g,:,:])
        node2vec = n2v.Node2Vec(graph, dimensions=n2vdim, walk_length=30, num_walks=200, workers=4)
        model = node2vec.fit(window=10, min_count=1, batch_words=4)
        graphdat = np.zeros((n2vdim, vertex))
        for i in range(vertex):
            graphdat[:,i] = model.wv.__contains__(str(i))
        #print(graphdat)
        n2vdata[g,:,:] = graphdat 
    return n2vdata 

In [ ]:
if  __name__ == "__main__":
    
    num_eg = 4000
    vertex = 36
    n2vdim = 20
    inputdim = vertex * vertex
    
    #train ensemble 
    #data = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    data = gs.generate_ensemble_v3(num_eg, vertex)
    """n2vdata = np.zeros((num_eg, n2vdim, vertex))
    for g in range(num_eg):
        graph = nx.from_numpy_matrix(data["graphs"][g,:,:])
        node2vec = n2v.Node2Vec(graph, dimensions=n2vdim, walk_length=30, num_walks=200, workers=4)
        model = node2vec.fit(window=10, min_count=1, batch_words=4)
        graphdat = np.zeros((n2vdim, vertex))
        for i in range(vertex):
            graphdat[:,i] = model.wv.__contains__(str(i))
        #print(graphdat)
        n2vdata[g,:,:] = graphdat 
    data["graphs"] = n2vdata """
    #test ensemble
    #testdata = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    testdata = gs.generate_ensemble_v3(num_eg, vertex)
    """n2vtestdata = np.zeros((num_eg, n2vdim, vertex))
    for g in range(num_eg):
        graph = nx.from_numpy_matrix(testdata["graphs"][g,:,:])
        node2vec = n2v.Node2Vec(graph, dimensions=n2vdim, walk_length=6, num_walks=20, workers=4)
        model = node2vec.fit(window=10, min_count=1, batch_words=4)
        graphdat = np.zeros((n2vdim, vertex))
        for i in range(vertex):
            graphdat[:,i] = model.wv.__contains__(str(i))
        #print(graphdat)
        n2vtestdata[g,:,:] = graphdat 
    testdata["graphs"] = n2vtestdata """
    
    #reformat 
    data["graphs"] = data["graphs"].reshape((num_eg, -1))
    testdata["graphs"] = testdata["graphs"].reshape((num_eg, -1))
    
    #Fully connected model 
    FullNN = FullNN(data["graphs"], data["labels"], testdata["graphs"], testdata["labels"], [inputdim,350,300,200,150,100,50,20,1])
    
    #Test Model 
    #FullNN.inputdat = testdata["graphs"]
    #FullNN.labeldat = testdata["labels"]
    #FullNN.trainmode = False
    #FullNN.evaluate()



epoch 500     |     cost 7.304228e-01     |     error train 38.450     |     error test 39.175
epoch 1000     |     cost 6.059466e-01     |     error train 16.200     |     error test 16.950
epoch 1500     |     cost 5.481846e-01     |     error train 6.150     |     error test 7.325
epoch 2000     |     cost 5.456473e-01     |     error train 5.525     |     error test 6.900
epoch 2500     |     cost 5.470282e-01     |     error train 6.025     |     error test 6.875
epoch 3000     |     cost 5.465832e-01     |     error train 5.875     |     error test 6.875
epoch 3500     |     cost 5.469857e-01     |     error train 5.900     |     error test 6.850
